# Coleta e preparação dos dados referentes aos roubos de aparelhos celular em São Paulo no primeiro semestre de 2023

## Coleta

Os dados que usaremos neste estudos foram coletados do site da Secretaria de Segurança Publica do governo de São Paulo. São dados públicos disponíveis em https://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx. 

O site apresenta um conjunto de temas cujos quais podemos pesquisar por dados do nosso interesse. Ao selecionar um tema, no nosso caso, "ROUBOS DE CELULAR",  uma tabela de dados com segmentada pelos anos e seus respectivos meses é apresentada. Logo abaixo da tabela, temos dois botões: O primero permite que baixemos a "METODOLOGIA", um arquivo de texto contendo informações sobre os dados e um dicionário de dados. O segundo exporta os dados em um formato `xls` (Excel).

Estou realizando a coleta em 08/2023, e destacaria três pontos de atenção para esta etapa:

1. O download dos dados demoram um tempo relativamente alto. Alguns downloads levaram mais de 3 minutos sem nenhum outro recurso utilizando uma banda de 200 MB.
2. O servidor cai com frequência. Tive que recarregar a página várias vezes pois um erro 500 ocorria toda vez que tentava fazer downloads consecutivos. Por esta razão, por enquanto, preferi não programar um crawler para obter os dados. 
3. Os dados vêm com um nome de arquivo inadequado: `DadosBO_2023_1(ROUBO DE CELULAR)`. Como o uso de parenteses espaços podem causar problemas, teremos que padronizar adequadamente o nome dos arquivos.

## Preparação dos dados 

Uma vez coletados os dados, a importação dos dados apresenta problemas:

In [1]:
import pandas as pd

Se tentarmos importar um arquivo para um DataFrame pandas, o seguinte erro é levantado:

In [13]:
file = "data/raw/DadosBO_SP/DadosBO_2023_1(ROUBO DE CELULAR).xls"
df_test = pd.read_excel(file)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

### Solucionado os problemas de importação

A exceção `ValueError: Excel file format cannot be determined, you must specify an engine manually` indica que formato do arquivo não pode ser determinado automaticamente. Essa exceção geralmente é lançada quando a `pandas` não consegue identificar o formato do arquivo do Excel. O erro também sugere que precisamos especificar manualmente o mecanismo (ou "engine") que o `pandas` deve usar para ler o arquivo do Excel. No nosso caso, como se trata de um arquivo `xls`, um modelo mais antigo da Microsoft, vamos utilizar a biblioteca `xlrd`, que é o padrão para este tipo de situação.

In [3]:
!conda install xlrd -y

Solving environment: done

# All requested packages already installed.



Utilizando a engine `xlrd`, obtemos o seguinte erro:

In [4]:
df_test = pd.read_excel(file, engine="xlrd")

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'

A exceção `XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'` ocorreu porque o `xldr` esperava encontrar um registro de início de arquivo ([BOF - Beginning of File](https://learn.microsoft.com/en-us/openspecs/office_file_formats/ms-xls/4d6a3d1e-d7c5-405f-bbae-d01e9cb79366)), mas encontrou algo diferente, representado como a sequência de bytes `b'A\x00N\x00O\x00_'`. Essa sequência de bytes pode ser uma pista para a natureza do problema, mas não é possível ter certeza sem mais contexto.

Ocorre que BOF é um termo usado para se referir ao registro inicial que marca o início de arquivos. É uma sequência específica de bytes que marca o começo de um arquivo e ajuda os programas a entenderem como interpretar e processar o conteúdo do arquivo, seja ele binário ou em outro formato. No nosso caso, o `xldr` esperava encontrar o BOF de um arquivo binário `xls` da Microsoft, mas encontrou outra sequência de bytes; o que pode indicar que o arquivo não é um `xls` mas um outro formato.

Uma forma de descobrir o formato do arquivo é identificar seu MIMETYPE. O tipo [MIME (Multipurpose Internet Mail Extensions)](https://en.wikipedia.org/wiki/MIME) é uma convenção para especificar o tipo de conteúdo de um arquivo com base em sua natureza e formato. Embora o MIME tenha sido projetado principalmente para [SMTP (Simple Mail Transfer Protocol)](https://pt.wikipedia.org/wiki/Simple_Mail_Transfer_Protocol), seus tipos de conteúdo também são importantes em outros protocolos de comunicação, como o [HTTP (HyperText Transfer Protocol)](https://pt.wikipedia.org/wiki/Hypertext_Transfer_Protocol), por exemplo. No HTTP para a web, os servidores inserem um campo de cabeçalho MIME no início de qualquer transmissão. Os clientes usam o tipo de conteúdo ou o cabeçalho do tipo de mídia para selecionar um aplicativo visualizador apropriado para o tipo de dados indicado.  

Antes de tentar abrir o arquivo em um aplicativo externo, vamos tentar utilizar o comando [`file`](https://manned.org/file.1) do UNIX passando a opção `-i` para tentar obter uma descrição detalhada e precisa do formato incluindo o tipo MIME: 


In [12]:
!file -i ./data/raw/DadosBO_SP/DadosBO_2023_1\(ROUBO\ DE\ CELULAR\).xls

 (main)./data/raw/DadosBO_SP/DadosBO_2023_1(ROUBO DE CELULAR).xls: application/octet-stream; charset=binary


A saída não é animadora: `application/octet-stream; charset=binary` indica que o arquivo foi identificado como um fluxo de bytes genérico (application/octet-stream) e que não foi possível determinar um charset (conjunto de caracteres) específico. Em outras palavras, utilitário `file` não conseguiu identificar com precisão o formato específico do arquivo e o tratou como um fluxo genérico de bytes. A identificação específica do formato pode ser desafiadora quando não há informações claras nos primeiros bytes do arquivo ou quando o formato é ambíguo.

Por esta razão, a alternativa mais rápida é tentar executá-lo em outra aplicação e observar como este aquivo é lido. As imagens abaixo mostram minha tentativa com LibreOffice:


![pop-up](./img/2023-08-28_00-49.png)
![pop-up](./img/2023-08-28_00-54.png)

Note que, ao tentar abrir o arquivo com LibreOffice Calc, este é lido como um `csv` e que o charset é "UTF-16". Na caixa de pré-visualização dos campos, da primeira imagem, vemos que os dados possuem uma `→` indicando que o separador é uma tabulação. A segunda imagem mostra que, ao selecionar "Tabulação" nas "Opções de separadores", os dados se organizam. Isso significa que, como desconfiávamos, se trata de outro tipo de formato. Não é novidade que tenhamos que lidar com situações desse tipo quando se trata de dados públicos governamentais. 

A próxima questão que queremos responder é: podemos carregar o conjunto de dados em um `DataFrame` do `pandas` usando o método `read_csv()` em vez de `read_excel()`? 

In [5]:
df_test = pd.read_csv(file, sep="\t", encoding="UTF-16")

UnicodeError: UTF-16 stream does not start with BOM

Sim, aparentemente é possível, contudo, obtemos um `UnicodeError` que indica um problema no conjunto de caracteres. Tentaremos compreender e solucionar esta exceção em seguida.

#### Compreendendo o formato de codificação de caracteres do conjunto de dados

A exceção `"UnicodeError: UTF-16 stream does not start with BOM` ocorre quando tentamos decodificar um fluxo de bytes, mas o fluxo de bytes não começa com o [BOM (Byte Order Mark)](https://en.wikipedia.org/wiki/Byte_order_mark) necessário. Em outras palavras, o decodificador do `pandas` esperava encontrar o BOM no início do fluxo de bytes UTF-16, mas não o encontrou. Isso geralmente ocorre quando o BOM foi omitido, ou quando o arquivo foi codificado em um formato diferente, mas erroneamente rotulado como UTF-16. Dado o que vimos até aqui, esta última possibilidade é bem plausível. Mas vamos continuar buscando pistas.

Para descobrir o esquema de codificação dos caracteres dos arquivos, vamos usar o utilitário [`enca`](https://linux.die.net/man/1/enca) do UNIX. O `enca` é um utilitário que detecta o conjunto de caracteres e a codificação de arquivos de texto e, também, pode convertê-los em outras codificações usando um conversor embutido ou bibliotecas externas como [libiconv](https://www.gnu.org/software/libiconv/), [librecode](https://ubuntu.pkgs.org/20.04/ubuntu-main-amd64/librecode-dev_3.6-24_amd64.deb.html) ou [cstocs](https://www.venea.net/man/Cz::Cstocs(3pm)).

Como meu objetivo é saber apenas o encoding, uso o seguinte comando passando um dos arquivos de dados:

In [11]:
!enca -L none ./data/raw/DadosBO_SP/DadosBO_2023_1\(ROUBO\ DE\ CELULAR\).xls

 (main)Universal character set 2 bytes; UCS-2; BMP
  LF line terminators
  Byte order reversed in pairs (1,2 -> 2,1)


A saída indica que o encoding é Universal Character Set de 2 bytes, ou UCS-2. Esse é um formato de codificação de caracteres que representa cada caractere em 2 bytes (16 bits). Os outros elementos da saída são os seguintes:

- **BMP** refere-se ao "[Basic Multilingual Plane](https://en.wikipedia.org/wiki/Plane_(Unicode)#Basic_Multilingual_Plane)", que é uma parte do conjunto de caracteres Unicode que abrange a maioria dos caracteres comumente usados em várias línguas. 
- **LF line terminators**, indica que o arquivo usa terminadores de linha **LF (Line Feed)**, que é um caractere de controle para indicar o fim de uma linha de texto. Isso é comum em sistemas baseados em Unix/Linux e é representado pelos caracteres `\n`. 
- **Byte order reversed in pairs (1,2 -> 2,1)** significa que a ordem dos bytes é invertida em pares (1, 2 -> 2, 1). 

Como dito, em UCS-2, cada caractere é representado por dois bytes. A ordem normal de armazenamento desses bytes na memoria seria primeiro o byte de ordem baixa (**Low Order Byte**) e depois o byte de ordem alta (**High Order Byte**). No entanto, com a ordem invertida em pares, os bytes são armazenados de forma reversa, ou seja, o byte de ordem alta vem antes do byte de ordem baixa.

Os conceitos de "Low Order Byte" (LOB) e "High Order Byte" (HOB) estão relacionados à forma como os bytes são organizados na memória em sistemas que utilizam múltiplos bytes para representar informações , como conjuntos de caracteres ou valores numéricos. Eles estão ligados a um conceito mais amplo conhecido como [Codificação de Largura Variável](https://pt.wikipedia.org/wiki/Codifica%C3%A7%C3%A3o_de_largura_vari%C3%A1vel), onde os mais comuns são as codificações multibyte, que usam vários números de bytes (octetos, daí a saída genérica "application/octet-stream" do comando `file`) para codificar diferentes caracteres. 

O LOB é o byte menos significativo em uma sequência de bytes que compõem uma unidade de informação. Em uma codificação de caracteres ou valor numérico, o LOB geralmente carrega a parte menos significativa da informação. Em contrapartida, o HOB é o byte mais significativo em uma sequência de bytes. Ele carrega a parte mais significativa da informação.

"Menos significativa" e "mais significativa" são terminologias relacionadas com a importância relativa de partes individuais de um valor binário (sequência de bits) ao representar informações. Só pra relembrar: 
- **Bit**: A menor unidade de informação em um sistema binário, podendo ser 0 ou 1.
- **Byte**: Um conjunto de 8 bits.
- **Sequência de Bytes**: Valores numéricos, caracteres e outros tipos de dados frequentemente representados por sequências de bytes.

Quando falamos sobre "menos significativa" e "mais significativa", estamos nos referindo à posição desses bits e bytes na representação binária de um valor. Essa terminologia se origina do sistema posicional que usamos para representar números, incluindo números decimais e binários.

**Exemplo para ficar menos abstrato:**

Em um sistema posicional, como o sistema decimal que usamos cotidianamente, o valor de um dígito depende de sua posição. Por exemplo, no número decimal "314", o "3" na posição das centenas representa uma quantidade significativamente maior do que o "1" na posição das dezenas, que é, por sua vez, mais significativo do que o "4" na posição das unidades.

Em uma representação binária funciona da mesma forma: cada bit, em uma posição específica, tem um valor que é uma potência de 2. Por exemplo, no número binário "10110", o bit mais à esquerda (o "1" mais significativo) representa 16 (ou $2^4$), enquanto o bit mais à direita (o "0" menos significativo) representa 1 (ou $2^0$).

Ao falar sobre partes "mais significativas" e "menos significativas", estamos observando como as posições dos dígitos ou bits contribuem para o valor geral de um número ou caractere. As partes mais significativas têm um impacto maior no valor total, enquanto as partes menos significativas têm um impacto menor.

Portanto, se trata de uma maneira de descrever a importância relativa das posições de dígitos ou bits em uma representação numérica ou binária, com base na forma como o sistema posicional funciona. Para ilustrar, imagine que estamos representando o número decimal 314 em uma codificação numérica de 16 bits:

```
00000001 00111010
```
Temos 16 bits divididos em dois bytes. A representação binária desse número é "0000000100111010". Agora, analisemos essa representação em termos de "menos significativa" e "mais significativa":

**Menos Significativa**: Os bits à direita são considerados menos significativos. No nosso exemplo, "00111010" é a parte menos significativa. Alterações nesses bits teriam um impacto menor no valor geral do número.

**Mais Significativa**: Os bits à esquerda são considerados mais significativos. No nosso exemplo, "00000001" é a parte mais significativa. Alterações nesses bits teriam um impacto maior no valor geral do número.

A saída do `enca` informa que a ordem dessas partes é invertida. Isso ocorre, pois a ordem em que os LOBs e HOBs são organizados pode variar entre diferentes sistemas e arquiteturas de computador. Existem dois principais padrões de ordenação:

**Little Endian**: Nesse padrão, o LOB é armazenado antes do HOB. Ou seja, o byte de menor valor é armazenado primeiro na memória. Isso é comum em muitos sistemas, incluindo os de arquitetura x86 e x86-64, que são amplamente usados em PCs e laptops.

**Big Endian**: Nesse padrão, o HOB é armazenado antes do LOB. O byte de maior valor é armazenado primeiro na memória. Isso é comum em algumas arquiteturas de processadores, como PowerPC e algumas implementações de redes.

Esses conceitos são particularmente relevantes ao lidar com codificações de caracteres em que cada caractere é representado por múltiplos bytes (multibyte). A ordem dos bytes pode afetar a forma como os caracteres são interpretados e, portanto, é importante considerar a ordem de bytes correta ao lidar com diferentes sistemas e formatos de dados. É justamente que estamos enfrentando no momento.  

#### Importando os dados com a codificação de caracteres correta

O UCS-2 é um esquema de codificação mais antigo e mais restrito porque só pode representar os caracteres do BMP, isto é, os caracteres Unicode que podem ser representados em 16 bits. Caracteres fora do BMP não podem ser representados diretamente em UCS-2. Verifiquei na  [documentação do `pandas`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) que a [lista de encodings suportados pelo Python](https://docs.python.org/3/library/codecs.html#standard-encodings) não inclui UCS-2. Por tanto, este é o motivo do `pandas`, não suportar esse encoding.

Contudo, o esquema **UTF-16**, uma codificação mais moderna e flexível que pode representar todos os caracteres Unicode, incluindo aqueles fora do BMP, também representa caracteres em 2 bytes e é suportada pela linguagem. Por que o `pandas` não foi capaz de carregar os dados mesmo assim? A resposta é simples: precisamos informar o decodificador do `pandas` que a ordem dos bytes está invertida, ou seja, precisamos indicar o padrão **Little Endian**, ou **"LE"**, para que a biblioteca consiga ler os bytes na ordem correta: 

In [15]:
df_test = pd.read_csv(file, sep="\t", encoding="UTF-16 LE")

In [16]:
df_test.head()

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,2023,2059,2059/2023,01/01/2023 00:08:34,01/01/2023 00:08:34,29/12/2022,NaN,A NOITE,30/12/2022,01/01/2023 00:08:34,Desconhecida,Não,NaN,Avenida Marechal Carmona,395.0,Vila Joao Jorge,CAMPINAS,SP,"-22,9181399","-47,0608299",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 3,05º D.P. CAMPINAS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Xiaomi
1,2023,27,27/2023,01/01/2023 00:39:51,01/01/2023 00:39:51,31/12/2022,23:32,A NOITE,01/01/2023,01/01/2023 00:39:51,Desconhecida,Não,NaN,Avenida Governador Mário Covas Júnior,10.0,Centro,PERUIBE,SP,"-24,3254647","-46,9961045",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DEL.POL.PERUIBE,DEL.POL.PERUIBE,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
2,2023,8583,8583/2023,01/01/2023 00:47:10,01/01/2023 00:47:12,30/01/2022,23:20,A NOITE,31/12/2022,01/01/2023 00:47:10,Desconhecida,Não,NaN,RUA BALDOMERO CARQUEJA,278.0,JD SÃO LUIS,S.PAULO,SP,"-23,6478994697931","-46,7509174963103",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,37º D.P. CAMPO LIMPO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
3,2023,8584,8584/2023,01/01/2023 00:47:35,01/01/2023 00:47:38,30/12/2022,22:05,A NOITE,31/12/2022,01/01/2023 00:47:35,Desconhecida,Não,NaN,"Avenida Antártica, 380",380.0,Água Branca,S.PAULO,SP,NaN,NaN,Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,23º D.P. PERDIZES,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
4,2023,8588,8588/2023,01/01/2023 00:48:47,01/01/2023 00:48:49,30/12/2022,20:30,A NOITE,31/12/2022,01/01/2023 00:48:47,Desconhecida,Não,NaN,Avenida Industrial,161.0,Jardim,S.ANDRE,SP,"-23,65238","-46,5300216",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,04º D.P. SANTO ANDRÉ,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Xiaomi


#### Padronizando o nome dos arquivos e importando os dados no Pandas

Agora que desvendamos a natureza dos arquivos de dados fornecidos pelo governo, vamos realizar as tarefas de transformar o nome dos arquivos para um padrão seguro e, finalmente, importar os datasets como o encoding e MIME corretos para gerar um `DataFrame` para proceguirmos.

O script a seguir, busca resolver ambos os problemas, padronizando o nome dos arquivos para "DadosBO_{ano}_{nome_do_mês}.csv, e realizando a leitura correta dos bytes gerando um DataFrame no qual podemos começar a trabalhar:

In [2]:
import os
import shutil
import re

In [3]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):

        if filename.endswith(".xls"):
            match = pattern.search(filename)

            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename).replace(".xls", ".csv")

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_20125/3893752474.py:46: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_20125/3893752474.py:46: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_20125/3893752474.py:46: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


### Solucionando os problemas de tipo dos dados

Agora que temos o tipo de arquivo correto e a importação dos dados foi realizada, observamos que, recebemos um aviso `DtypeWarning`. Vamos compreender a razão pela qual este aviso ocorre e checar como podemos processar os dados adequadamente. 

#### Compreendendo DtypeWarning

A saída `DtypeWarning` do comando de importação nos avisa que o `pandas` não conseguiu identificar o tipo de algumas colunas do dataset e, por isso, teve que inferir um tipo. Em outras palavras, o `pandas` tenta determinar os tipos de cada coluna; uma operação que exige muita memória.  

Podemos silenciar esses avisos basicamente de duas formas: A primeira é utilizando o parâmetro `dtypes` e passando `object` como valor.`Object` [é a classe base de onde derivam toda as classes em Python](https://docs.python.org/pt-br/3/library/functions.html?highlight=object#object), ou seja, `object` pode conter qualquer objeto Python. Isso fará com que o `pandas` defina o tipo `object` para todos os dados. Esse tipo já é aplicado para tipos de dados mistos ou que o `pandas` não consegue inferir. [No entanto, essa medida deve ser evitado na medida do possível](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) (para desempenho e interoperabilidade com outras bibliotecas e métodos).

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                dtypes=object) 
                for file in file_path_genarator]
```

A outra forma é usar o parâmetro `low-memory=False`, como sugere o próprio aviso: `DtypeWarning: Columns (...,...) have mixed types. Specify dtype option on import or set low_memory=False`.

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                low-memory=True) 
                for file in file_path_genarator]
```
 Porém, esse parâmetro, embora funcione, é considerado obsoleto e pode levar a erros silenciosos que não são documentados. [Existe uma discussão que se arrasta desde 2014 sobre se este parâmetro deve ou não ser melhor documentado ou retirado da biblioteca](https://github.com/pandas-dev/pandas/issues/5888). O fato é que sua utilização não é muito comum apesar do aviso indicar seu uso.

De toda forma, nenhuma das abordagens resolvem o problema de fato. O `pandas` continuará usando muito recurso para adivinhar os tipos. 

Portanto, a abordagem mais indicada nesses casos é fornecer os tipos manualmente. 

Antes de partir para esta tarefa, vamos checar o estado do DataFrame:

In [19]:
df.shape

(119158, 54)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o

A saída do método `info()` nos fornece as seguintes informações:

**Column**: Esta coluna indica os nomes das colunas presentes no DataFrame.

**Non-Null Count**: Esta coluna mostra a contagem de valores não nulos em cada coluna. Em outras palavras, é o número de entradas que possuem valores válidos para aquela coluna. Por exemplo, na primeira coluna "ANO_BO", existem 119158 valores não nulos.

**Dtype**: Esta coluna informa o tipo de dados (data type) presente em cada coluna. Por exemplo, `object`, como vimos, indica que os registros podem qualquer objeto python, geralmente são strings ou valores mistos. Manter os dados assim pode causar inúmeros problemas já que strings e não-strings podem estar juntas em uma mesma série. Trabalhar com os tipos corretos no DataFrame é extremamente importante para que possamos garantir:

- Precisão nos dados
- Economia de memória
- Desempenho
- Manipulação dos dados mais eficaz
- Prevenção de erros
- Consistência. 

Podemos notar que, com exceção de alguns tipos `float64`, o pandas inferiu `object` em todo o conjunto de dados. Vamos olhar uma amostra aleatória dos registros para termos uma noção da tipagem que precisará ser adequada.

In [21]:
df.sample(10)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
36180,2023,84427,84427/2023,02/06/2023 11:12:43,02/06/2023 11:12:43,31/05/2023,15:40,A TARDE,02/06/2023,02/06/2023 11:12:43,Desconhecida,Não,NaN,RUA JOSÉ GARCÍA DE SOUZA,1110.0,JD IMPERADOR,SUZANO,SP,"-23,531961367","-46,31511331",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 2,DEL.POL.SUZANO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
107190,2023,80045,80045/2023,17/03/2023 13:49:30,17/03/2023 13:49:30,17/03/2023,04:00,DE MADRUGADA,17/03/2023,17/03/2023 13:49:30,Desconhecida,Não,NaN,AVENIDA ANGELO CRISTIANINI,1434.0,CIDADE ADEMAR,S.PAULO,SP,"-23,6905418953384","-46,6437659202769",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 1,98º D.P. JARDIM MIRIAM,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Outros
28099,2023,1738,1738/2023,22/05/2023 13:20:04,22/05/2023 13:20:04,19/05/2023,20:30,A NOITE,20/05/2023,22/05/2023 13:20:04,Desconhecida,Não,1719/2023 - 30406,NaN,0.0,POTUVERA,ITAPECERICA DA SERRA,SP,NaN,NaN,Condominio Residencial,NaN,BO PARA INVESTIGAÇÃO,DEL.POL.ITAPECERICA DA SERRA,DEL.POL.ITAPECERICA DA SERRA,Localização e/ou Devolução,Localização/Apreensão e Entrega de veículo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRW3823,SP,ITAPECERICA DA SERRA,Cinza,I/FORD RANGER XLT CD2 25,2014.0,2014.0,CAMINHONETE,1.0,Apple
112292,2023,653,653/2023,23/03/2023 12:59:44,23/03/2023 12:59:44,22/03/2023,11:00,PELA MANHÃ,22/03/2023,23/03/2023 12:59:44,Conhecida,Sim,647/2023 - 30440,Rua Alfazema,99.0,Jardim Munhoz Júnior,OSASCO,SP,"-23,5020368","-46,8099188",Via pública,NaN,BO PARA FLAGRANTE,10º D.P. OSASCO,10º D.P. OSASCO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - CARGA,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.0,Samsung
2692,2023,5651,5651/2023,04/01/2023 13:17:26,04/01/2023 13:17:26,02/01/2023,14:00,A TARDE,03/01/2023,04/01/2023 13:17:26,Desconhecida,Não,NaN,NaN,0.0,VILA XAVIER,SALTO DE PIRAPORA,SP,NaN,NaN,Residência,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 3,DEL.POL.SALTO DE PIRAPORA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - RESIDENCIA,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
53972,2023,1011,1011/2023,28/06/2023 05:26:10,28/06/2023 05:26:10,27/06/2023,22:00,A NOITE,28/06/2023,28/06/2023 05:26:10,Conhecida,Sim,NaN,RODOVIA SP 019,999.0,AEROPORTO,GUARULHOS,SP,"-23,454168196","-46,490361283",Via pública,NaN,BO PARA FLAGRANTE,03º D.P. AEROP/TUR-GUARULHOS,03º D.P. AEROP/TUR-GUARULHOS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Motorola
53769,2023,2182,2182/2023,27/06/2023 18:48:59,27/06/2023 18:48:59,27/06/2023,18:10,A NOITE,27/06/2023,27/06/2023 18:48:59,Desconhecida,Não,NaN,RUA MOGI GUASSU,1.0,VILA SANTA PAULA,S.CAETANO DO SUL,SP,"-23,626202034","-46,5602359329999",Via pública,NaN,APRECIAÇÃO DO DELEGADO TITULAR,DEL.POL.S.CAETANO DO SUL,02º D.P. S.CAETANO DO SUL,Título II -

Ao observar os registros do DataFrame, podemos notar o seguinte:

1. Existem inconsistências nos registros: Além das datas estarem em formato inadequado para a maioria dos softwares e fora do padrão para dados,  alguns deles estão com caracteres maiúsculos e outros minúsculos. Será necessário padronizar os tipos de data e as strings e limpar os espaços extras. 
2. Os registros de `LATITUDE` e `LONGITUDE` possuem um espaço extra tanto à esquerda quanto à direita. Será necessário limpar estes espaços, substituir a "," por "." e converter o dados adequadamente para `float`.
3. A única coluna que se refere a valores do tipo `int` e que pode ser utilizada para cálculo é "QUANT_CELULAR", que se refere a quantidade de dispositivos roubados da vítima. Esta variável precisa ser convertida para o tipo adequado que suporte valores ausentes.
4. Existem muitos valores ausentes. Será necessário pensar em uma abordagem para lidar com eles.

Antes de partir para solução desses problemas, é necessário que tenhamos mais informações sobre o dataset como os significado de cada uma das variáveis. Conhecer o máximo possível é vital para que saibamos como proceguir com a exploração dos dados. Principalmente quando estamos lidando com dados públicos. Neste momento, o dicionário de dados é a melhor ferramenta.

#### Conferindo o dicionário de dados

Como uma tarefa complementar desta etapa, para facilitar a comparação entre o dicionário e o DataFrame, eu exportei a saída da propriedade `dtypes` e copiei a tabela do documento "METODOLOGIA" para uma planilha a fim de:

1. Checar se o DataFrame contém todas as variáveis presentes no dicionário de dados;
2. Identificar os tipos dos dados e atribuí-los adequadamente ao DataFrame.

A planilha se encontra no diretório `data` e, como poderá ser observado, fiz uma busca vertical (VLOOKUP ou PROCV) para cruzar as colunas que se referem às características do conjunto de dados. O código da exportação dos dtypes segue abaixo:

In [22]:
t = df.dtypes.reset_index()
t.columns = ["Variáveis", "Tipos"]
t.to_excel("./data/dtypes_out.xlsx", index=False)

Ao analisar o dicionário de dados, cheguei à conclusão de que ele oferece pouca ajuda. Este é outro problema quando trabalhamos com dados públicos. Os motivos são destacados a seguir:

1. O dicionário não informa os tipos dos dados;
2. As descrições das variáveis são vagas e não fornecem mais informações além do óbvio;
3. Os rótulos das variáveis especificadas no dicionário são diferentes dos rótulos do DataFrame, mas algumas se referem à mesma coisa. Por exemplo: `BOLETIM_EMITIDO`, uma das variáveis do DataFrame, não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO" definida apenas como "Data-hora da elaboração da impressão" no dicionário.
4. O DataFrame possui 54 variáveis enquanto o dicionário indica 62. Dada a condição anterior, não consigo afirmar com certeza se o DataFrame possui variáveis não especificadas no dicionário de dados ou se as variáveis do dicionário se referem exclusivamente às do DataFrame.
5. O dicionário de dados informa que, várias linhas podem se referir ao mesmo boletim. Portanto, segundo o documento, para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades das seguintes variáveis:

- NOME_DELEGACIA
- ANO_BO
- NUM_BO

Neste cenário, podemos seguir com uma verificação básica por dois caminhos a princípio:

1. Verificação do formato dos dados e seus tipos adequados;
2. Verificação da integridade básica: limpeza e as duplicidades indicadas;

#### Classificando os dados

Com pouca (ou quase nenhuma informação) sobre a tipagem dos dados, o que nos resta fazer é checar cada variável do dataset, identificar as variáveis categóricas e núméricas e inferir o tipo mais adequado levando em consideração a semântica da variável.  

**Características Categóricas**:

- As características categóricas são variáveis que representam categorias ou grupos discretos. Elas não possuem uma relação natural de ordem entre as categorias. Em outras palavras, não é possível dizer que uma categoria é "maior" ou "menor" do que outra. Exemplos no nosso dataset são: nomes, estados civis, bairro, etc. Elas são subdivididas em duas categorias: nominais e ordinais.

    - Características categóricas nominais: Não existe uma ordem intrínseca nas categorias. Exemplos no dataset incluem, além das datas do roubo, elaboração e registro do boletim, `MARCA_CELULAR`, `ESTADOCIVIL`, `DESCRICAOLOCAL`, `PROFISSÃO`, `PERIDOOCORRENCIA`.
        
    - Características categóricas ordinais: As categorias têm uma ordem específica, mas a diferença entre elas pode não ser uniforme. Por exemplo, `FLAGRANTE`, cujo o valor pode ser "SIM"/"NÃO", da mesma forma a variável `STATUS` que, nesse caso, é valorado com os valores "Consumado" ou "Tentado", são ordinais.

**Características Numéricas**:

- As características numéricas representam valores numéricos que podem ser quantificados e têm uma relação de ordem natural. Elas podem ser subdivididas em duas categorias principais: discretas e contínuas.
    - Características numéricas discretas: Representam valores inteiros ou contáveis. No dataset, o exemplo mais evidente é `QUANT_CELULAR`, que contabiliza a quantidade de aparelhos roubados registrados no boletim.
    - Características numéricas contínuas: Representam valores em um intervalo contínuo e podem assumir qualquer valor dentro desse intervalo. No nosso dataset, os dados geoespaciais `LATITUDE` e `LONGITUDE` pode ser considerados contínuos posi podem ser usados em cálculos de distância entre pontos geográficos, densidade populacional em uma área, áreas de polígonos geográficos, e outras análises que requerem operações matemáticas. Contudo, elas também podem ser consideradas Categóricas, já que também podem ser usadas para categorizar regiões geográficas específicas. Portanto, esse tipo de dado, pode ser considerado Contínuo ou Categórico a depender da escolha de como serão representados. 

Para formatar os dados corretamente, vamos considerar as variáveis categóricas como `strings`, com exceção das datas, que serão formatados como o tipo `Date` adequado, e as variáveis numéricas, formataremos como tipos núméricos expecíficos, como `int` e `float`.


#### Formatando os dados de acordo com suas características

O método `read_csv()` possui diversos argumentos nomeados que podemos fornecer para cuidar da formatação dos registros. Estes argumentos precisam ser estruturas de dados específicas:
- `dtype`: Precisa ser um tipo, como `str`, ou um dicionário. Ele define os tipos de dados a serem aplicados a todo o conjunto de dados ou a colunas individuais
- `parse_dates`: `bool`, lista de Hashable, lista de listas ou dict de {Hashable list}, padrão `False`.  
- `date_format`: `str` ou `dict`  com nomes das colunas e um formato opcional
- `converters`: `dict` de {Hashable Callable}, optional. Aqui tem um peculiaridade: Se `converters` for definido, ele terá prioridade em vez do `dtype`.

Antes de definir as estruturas, vamos criar uma função para limpar e converter os dados de `LONGITUDE` e `LATITUDE` para `float`. Atualmente os dados dessas variáveis estão em um formato que não permite que trablhemos com elas. Elas possuem caractéres de espaços tanto à esquerda quanto à direita. Além disso, há uma "," como ponto flutuante, vamos substituir para ".". Por fim, convertemos para o tipo correto:

In [4]:
"""Limpa os registro LONGITUDE e LATITUDE """

def clean_coordinate_value(value):
    if isinstance(value, str):
        try:
            clean_value = value.strip().replace(",", ".")
            return float(clean_value)
        except ValueError:
            return None 
        
    return None

Agora, podemos definir as estruturas que iremos passar para `read_csv()`:

In [5]:
DTYPES = {
    "ANO_BO": pd.StringDtype(),
    "ANO_FABRICACAO": pd.StringDtype(),
    "ANO_MODELO": pd.StringDtype(),
    "BO_AUTORIA": pd.StringDtype(),
    "CIDADE": pd.StringDtype(),
    "BAIRRO": pd.StringDtype(),
    "LOGRADOURO": pd.StringDtype(),
    "CIDADE_VEICULO": pd.StringDtype(),
    "CORCUTIS": None,
    "DELEGACIA_CIRCUNSCRICAO": pd.StringDtype(),
    "DELEGACIA_NOME": pd.StringDtype(),
    "DESCR_COR_VEICULO": pd.StringDtype(),
    "DESCR_MARCA_VEICULO": pd.StringDtype(),
    "DESCR_TIPO_VEICULO": pd.StringDtype(),
    "DESCRICAOLOCAL": pd.StringDtype(),
    "DESDOBRAMENTO": pd.StringDtype(),
    "ESPECIE": pd.StringDtype(),
    "ESTADOCIVIL": pd.StringDtype(),
    "EXAME": None,
    "FLAGRANTE": pd.StringDtype(),
    "GRAUINSTRUCAO": pd.StringDtype(),
    "IDADE": pd.Int64Dtype(),
    "MARCA_CELULAR": pd.StringDtype(),
    "NACIONALIDADE": pd.StringDtype(),
    "NATURALIDADE": pd.StringDtype(),
    "NATUREZAVINCULADA": pd.StringDtype(),
    "NUM_BO": pd.StringDtype(),
    "NUMERO": pd.StringDtype(),
    "NUMERO_BOLETIM": pd.StringDtype(),
    "NUMERO_BOLETIM_PRINCIPAL": pd.StringDtype(),
    "PARENTESCO": pd.StringDtype(),
    "PERIDOOCORRENCIA": pd.StringDtype(),
    "PLACA_VEICULO": pd.StringDtype(),
    "PROFISSAO": pd.StringDtype(),
    "QUANT_CELULAR": pd.Int64Dtype(),
    "RELACIONAMENTO": pd.StringDtype(),
    "RUBRICA": pd.StringDtype(),
    "SEXO": pd.StringDtype(),
    "SOLUCAO": pd.StringDtype(),
    "STATUS": pd.StringDtype(),
    "TIPOPESSOA": pd.StringDtype(),
    "TIPOVINCULO": pd.StringDtype(),
    "UF": pd.StringDtype(),
    "UF_VEICULO": pd.StringDtype(),
    "VITIMAFATAL": pd.StringDtype(),
}

DATE_TYPES = [
    "BO_EMITIDO",
    "BO_INICIADO",
    "DATACOMUNICACAO",
    "DATAELABORACAO",
    "DATANASCIMENTO",
    "DATAOCORRENCIA",
]

DATE_FORMAT = {
    "BO_EMITIDO": "%d/%m/%Y %H:%M:%S",
    "BO_INICIADO": "%d/%m/%Y %H:%M:%S",
    "DATACOMUNICACAO": "%d/%m/%Y",
    "DATAELABORACAO": "%d/%m/%Y %H:%M:%S",
    "DATANASCIMENTO": "%d/%m/%Y",
    "DATAOCORRENCIA": "%d/%m/%Y",
    "HORAOCORRENCIA": "%H:%M",
}

CONVERTERS = {
    "LATITUDE": clean_coordinate_value,
    "LONGITUDE": clean_coordinate_value,
}

TO_UPPER_LIST = [
    "ANO_BO",
    "ANO_FABRICACAO",
    "ANO_MODELO",
    "BO_AUTORIA",
    "CIDADE",
    "BAIRRO",
    "LOGRADOURO",
    "CIDADE_VEICULO",
    "DELEGACIA_CIRCUNSCRICAO",
    "DELEGACIA_NOME",
    "DESCR_COR_VEICULO",
    "DESCR_MARCA_VEICULO",
    "DESCR_TIPO_VEICULO",
    "DESCRICAOLOCAL",
    "DESDOBRAMENTO",
    "ESPECIE",
    "ESTADOCIVIL",
    "FLAGRANTE",
    "GRAUINSTRUCAO",
    "MARCA_CELULAR",
    "NACIONALIDADE",
    "NATURALIDADE",
    "NATUREZAVINCULADA",
    "NUM_BO",
    "NUMERO",
    "NUMERO_BOLETIM",
    "NUMERO_BOLETIM_PRINCIPAL",
    "PARENTESCO",
    "PERIDOOCORRENCIA",
    "PLACA_VEICULO",
    "PROFISSAO",
    "RELACIONAMENTO",
    "RUBRICA",
    "SEXO",
    "SOLUCAO",
    "STATUS",
    "TIPOPESSOA",
    "TIPOVINCULO",
    "UF",
    "UF_VEICULO",
    "VITIMAFATAL"
]

Poderíamos passar `str` para os campos do dict `DTYPES` referentes a strings, contudo, ainda teríamos registros avaliados como `object`. Portanto, a [maneira indicada de trabalharmos com registros de texto no `pandas` é utilizando a classe `StringDtype()`](https://pandas.pydata.org/docs/user_guide/text.html#string-methods). 

Outro ponto importante de destacar é a utilização da classe `Int64Dtype()` utilizada para a variável "QUANT_CELULAR". Usar `int`, neste caso, resultaria na exceção `ValueError: invalid literal for int() with base 10` que indica que o `pandas` não consegue converter uma string em um número. Essa confusão acontece, pois o `pandas` interpretou a variável como `object` para poder suportar valores nulos. Como a função embutida `int` é intolerante e levanta uma exceção quando esbarra com um valor que não consegue converter (como valores nulos), precisamo usar `Int64Dtype()` que é um tipo de numeral inteiro anulável. 

Por fim, a constante `TO_UPPER_LIST` será utilizada para transformar os caracteres minúsculos em maiúsculo sem que o tipo da `Serie` volte a ser `object`. Isso garante mais consistência.


In [6]:
"""
Padroniza os nomes dos arquivos para "DadosBO_SP_{ano}_{mes}.xls
"""
def process_file(input_dir, output_dir):

    PATTERN = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")

    MONTHS = {
        "1": "Janeiro",
        "2": "Fevereiro",
        "3": "Março",
        "4": "Abril",
        "5": "Maio",
        "6": "Junho",
        "7": "Julho",
        "8": "Agosto",
        "9": "Setembro",
        "10": "Outubro",
        "11": "Novembro",
        "12": "Dezembro"
    }

    for filename in os.listdir(input_dir):
        # Verificar se o arquivo é um arquivo xls
        if filename.endswith(".xls"):
            # Pesquisar o padrão
            match = PATTERN.search(filename)
            # Se verdadeiro, capturar o dígito da string e utilizá-lo para obter o valor da chave do dict `meses`
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                # Novo nome do arquivo após substituição
                new_name = PATTERN.sub(f"{name_month}", filename).replace(".xls", ".csv")

                # Caminhos completos dos arquivos antigo e novo
                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

file_path_generator = process_file(input_dir, output_dir)

# Passa as estruturas de dados pra read_csv realizar as operações na importação:
datasets = [pd.read_csv(file, sep='\t',
                        encoding="UTF-16 LE",
                        dtype=DTYPES,
                        parse_dates=DATE_TYPES,
                        date_format=DATE_FORMAT,
                        converters=CONVERTERS)
            for file in file_path_generator]

df_formated = pd.concat(datasets, axis=0, ignore_index=True)

for c in TO_UPPER_LIST:
    df_formated[c] = df_formated[c].str.upper()
    df_formated.columns.str.strip()


pd.set_option("display.max_columns", None)

In [56]:
df_formated.shape

(119158, 54)

In [57]:
df_formated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

Observe que agora temos cada variável com seu tipo adequado. Abaixo podemos observar que os dados estão mais consistentes, com as datas no formato padrão para bancos de dados (YYYY-MM-DD) e o valor das variáveis categóricas em letras maíusculas.

In [58]:
df_formated.head()

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,2023,2059,2059/2023,2023-01-01 00:08:34,2023-01-01 00:08:34,2022-12-29,NaN,A NOITE,2022-12-30,2023-01-01 00:08:34,DESCONHECIDA,NÃO,<NA>,AVENIDA MARECHAL CARMONA,395,VILA JOAO JORGE,CAMPINAS,SP,-22.918140,-47.060830,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 3,05º D.P. CAMPINAS,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,XIAOMI
1,2023,27,27/2023,2023-01-01 00:39:51,2023-01-01 00:39:51,2022-12-31,23:32,A NOITE,2023-01-01,2023-01-01 00:39:51,DESCONHECIDA,NÃO,<NA>,AVENIDA GOVERNADOR MÁRIO COVAS JÚNIOR,10,CENTRO,PERUIBE,SP,-24.325465,-46.996105,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DEL.POL.PERUIBE,DEL.POL.PERUIBE,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
2,2023,8583,8583/2023,2023-01-01 00:47:10,2023-01-01 00:47:12,2022-01-30,23:20,A NOITE,2022-12-31,2023-01-01 00:47:10,DESCONHECIDA,NÃO,<NA>,RUA BALDOMERO CARQUEJA,278,JD SÃO LUIS,S.PAULO,SP,-23.647899,-46.750917,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,37º D.P. CAMPO LIMPO,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
3,2023,8584,8584/2023,2023-01-01 00:47:35,2023-01-01 00:47:38,2022-12-30,22:05,A NOITE,2022-12-31,2023-01-01 00:47:35,DESCONHECIDA,NÃO,<NA>,"AVENIDA ANTÁRTICA, 380",380,ÁGUA BRANCA,S.PAULO,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,23º D.P. PERDIZES,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
4,2023,8588,8588/2023,2023-01-01 00:48:47,2023-01-01 00:48:49,2022-12-30,20:30,A NOITE,2022-12-31,2023-01-01 00:48:47,DESCONHECIDA,NÃO,<NA>,AVENIDA INDUSTRIAL,161,JARDIM,S.ANDRE,SP,-23.652380,-46.530022,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,04º D.P. SANTO ANDRÉ,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,XIAOMI


Concluindo a etapa de formatação dos dados, também solucionamos os problema dos tipos dos dados. Vamos passar para etapa de limpeza do dataset. 

## Limpeza dos dados

Nesta etapa, iremos checar a integridade básica dos dados, eliminar linhas de qualidade duvidosa, checar duplicatas, buscar uma maneira de lidar com valores ausentes.

In [35]:
df_formated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

Verificar as duplicidades de 

- NOME_DELEGACIA
- ANO_BO
- NUM_BO

In [7]:
num_bo_counts = df_formated["NUM_BO"].value_counts()
num_bo_counts.sort_values(ascending=False)


NUM_BO
1011      193
1252      175
219       161
218       157
1379      153
         ... 
906221      1
906224      1
906236      1
906239      1
507554      1
Name: count, Length: 53335, dtype: Int64

In [8]:
# Quantas ocorrências de um mesmo registro (47354 registros ocorrem apenas uma vez)
num_bo_counts.value_counts()

count
1      47354
2       1812
4        652
3        309
6        230
       ...  
71         1
175        1
63         1
62         1
193        1
Name: count, Length: 101, dtype: Int64

Observe os resultados acima: existem 119158 registros no dataset. 53335 são registros únicos cujos quais, 47354 ocorrem apenas uma vez, restando 5981 registros do dataset que, na verdade são   que ocorrem mais de uma vez.  
Queremos os índices da série `num_bo_counts` cuja contagem é > 1 para localizar as duplicidades. 

In [11]:
dupe_bo_mask = num_bo_counts > 1
dupe_bo_mask.value_counts()

count
False    47354
True      5981
Name: count, dtype: Int64

In [42]:
dupe_bo_mask[0:5]

NUM_BO
1011    True
1252    True
219     True
218     True
1379    True
Name: count, dtype: boolean

In [66]:
dupe = df_formated[df_formated.duplicated("NUM_BO", keep=False)]
dupe.sort_values(ascending=False, by="NUM_BO")

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
10908,2023,99968,99968/2023,2023-01-18 23:23:41,2023-01-18 23:23:46,2023-01-18,15:50,A TARDE,2023-01-18,2023-01-18 23:23:41,DESCONHECIDA,NÃO,<NA>,RUA GAMA LOBO,1300,VILA DOM PEDRO I,S.PAULO,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,17º D.P. DOUTOR ALDO GALIANO,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,DKU1662,SP,S.PAULO,BRANCO,FIAT/TORO ENDURANCE AT6,2019,<NA>,AUTOMOVEL,1,SAMSUNG
114024,2023,99968,99968/2023,2023-03-25 19:18:34,2023-03-25 19:18:34,2023-03-24,21:40,A NOITE,2023-03-25,2023-03-25 19:18:34,DESCONHECIDA,NÃO,<NA>,RUA CUIABÁ,431,VILA AMELIA,RIBEIRAO PRETO,SP,-21.160128,-47.830391,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 3,03º D.P. RIBEIRAO PRETO,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA
75437,2023,999,999/2023,2023-04-04 12:30:02,2023-04-04 12:30:02,2023-04-03,NaN,PELA MANHÃ,2023-04-04,2023-04-04 12:30:02,DESCONHECIDA,NÃO,992/2023 - 30102,AVENIDA LAURO GOMES,2000,VILA PRÍNCIPE DE GALES,S.ANDRE,SP,-23.665633,-46.553961,VIA PÚBLICA,NaN,BO PARA INVESTIGAÇÃO,01º D.P. SANTO ANDRÉ,04º D.P. SANTO ANDRÉ,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA
75433,2023,999,999/2023,2023-04-04 12:30:02,2023-04-04 12:30:02,2023-04-03,NaN,PELA MANHÃ,2023-04-04,2023-04-04 12:30:02,DESCONHECIDA,NÃO,992/2023 - 30102,AVENIDA LAURO GOMES,2000,VILA PRÍNCIPE DE GALES,S.ANDRE,SP,-23.665633,-46.553961,VIA PÚBLICA,NaN,BO PARA INVESTIGAÇÃO,01º D.P. SANTO ANDRÉ,04º D.P. SANTO ANDRÉ,LOCALIZAÇÃO E/OU DEVOLUÇÃO,LOCALIZAÇÃO/APREENSÃO E ENTREGA DE VEÍCULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA
30526,2023,999,999/2023,2023-05-25 15:23:29,2023-05-25 15:23:29,2023-05-24,18:30,A NOITE,2023-05-25,2023-05-25 15:23:29,DESCONHECIDA,NÃO,<NA>,AVENIDA LINS DE VASCONCELOS,431,CAMBUCI,S.PAULO,SP,-23.567075,-46.621456,VIA PÚBLICA,NaN,APRECIAÇÃO DO DELEGADO TITULAR,06º D.P. CAMBUCI,06º D.P. CAMBUCI,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,2023,1,1/2023,2023-01-02 14:31:43,2023-01-02 14:31:43,2022-10-20,05:00,DE MADRUGADA,2022-10-20,2023-01-02 14:31:43,CONHECIDA,SIM,2689/2022 - 30929,AVENIDA IBIRAPITANGA,377,VILA PIRES,S.ANDRE,SP,-23.677494,-46.512863,VIA PÚBLICA,NaN,BO PARA FLAGRANTE,CPJ SANTO ANDRE,03º D.P. SANTO ANDRÉ,EXCLUDENTES DE ILICITUDE - CPB,"ESTADO DE NECESSIDADE (ART. 23, I)",<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,XIAOMI
1038,2023,1,1/2023,2023-01-02 14:31:43,2023-01-02 14:31:43,2022-10-20,05

In [62]:
dupe_2 = df_formated[df_formated.duplicated()]
dupe_2

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR


## Referências
- [Microsoft - BOF (Beginning of File)](https://learn.microsoft.com/en-us/openspecs/office_file_formats/ms-xls/4d6a3d1e-d7c5-405f-bbae-d01e9cb79366)
- [Wikipedia - BOM (Byte Order Mark)](https://en.wikipedia.org/wiki/Byte_order_mark)
- [Wikipedia - BPM (Basic Multilingual Plane)](https://en.wikipedia.org/wiki/Plane_(Unicode)#Basic_Multilingual_Plane)
- [Wikipedia - MIME (Multipurpose Internet Mail Extensions)](https://en.wikipedia.org/wiki/MIME)
- [Wikipedia - Codificação de Largura Variável](https://pt.wikipedia.org/wiki/Codifica%C3%A7%C3%A3o_de_largura_vari%C3%A1vel)
- [Unicode Consortium - Glossary](https://www.unicode.org/glossary/)
- [UTF-8 and Unicode FAQ for Unix/Linux](https://www.cl.cam.ac.uk/~mgk25/unicode.html)
- [Wikipedia - Universal Character Set](https://en.wikipedia.org/wiki/Universal_Coded_Character_Set)
- [Wikipedia - UTF-16](https://en.wikipedia.org/wiki/UTF-16)
- [Wikipedia - Endianness](https://en.wikipedia.org/wiki/Endianness)
- [Documentação Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
- [Documentação Python](https://docs.python.org/3/library/codecs.html#standard-encodings)